<a href="https://colab.research.google.com/github/Zolinad/Proc_Imag_-_Vis_Comp/blob/main/AULA10/treinar_modelo_cvkeras_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Treinamento de Rede Neural Convolucional para Reconhecimento e Classificação de Faces**

In [ ]:
# O dataset deve ser zipado e ser arrastado para o Colab
#
# O dataset zipado deve ser descompactado na pasta local do Colab com o seguinte comando:
!unzip dataset.zip -d 

In [6]:
# Importando bibliotecas necessárias
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np

# Definindo o gerador de dados (DIVERSIFICANDO os dados, ao variar as imagens)
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalização
    validation_split=0.2,   # Separar 20% para validação (sobram 80% para treino)
    height_shift_range=0.2, # Transladar verticalmente até 20% da altura
    width_shift_range=0.2,  # Transladar horizontalmente até 20% da largura
    rotation_range=20,      # Rotacionar imagens aleatoriamente até 20 graus
    zoom_range=0.2,         # Aplicar zoom aleatório
    horizontal_flip=True,   # Inverter imagens horizontalmente
)

# Carregar os dados de treino
train_data = datagen.flow_from_directory(
    'dataset',        # Caminho para o dataset
    target_size = (128, 128),  # Redimensionar as imagens
    batch_size = 128,          # Número de amostras por batch
    class_mode ='categorical', # Classificação multiclasses
    subset = 'training',       # Conjunto de treinamento
)

# Carregar os dados de validação
val_data = datagen.flow_from_directory(
    'dataset',                 # Mesmo caminho, já que é o mesmo dataset
    target_size = (128, 128),   # Redimensionar as imagens
    batch_size = 128,           # Tamanho do batch
    class_mode = 'categorical', # Classificação multiclasses
    subset = 'validation'       # Conjunto de validação
)

# Construindo o modelo
model = Sequential()

K = 3  # Tamanho do kernel
P = 2  # Pooling size

# Primeira camada de convolução + pooling
model.add(Conv2D(32, (K, K), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Segunda camada de convolução + pooling
model.add(Conv2D(64, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Terceira camada de convolução + pooling
model.add(Conv2D(128, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Quarta camada de convolução (opcional)
model.add(Conv2D(256, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Quinta camada de convolução (opcional)
model.add(Conv2D(512, (K, K), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(P, P)))

# Camada de achatamento
model.add(Flatten())

# Exibir o resumo do modelo antes de adicionar a camada densa
model.summary()

# Quantidade de neurônios na camada densa (de acordo com a saída na camada de achatamento (Flatten))
camada_densa = model.layers[-1].output.shape[1]   # Acessa a forma da saída através do atributo 'output'
print(f"Número de neurônios da camada densa: {camada_densa}")

# Deep Learning - Várias camadas densas dentro da camada oculta da Rede Neural (fully connected)
model.add(Dense(camada_densa, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))

# Camada de saída com 20 classes (softmax para classificação multiclasses)
model.add(Dense(20, activation='softmax'))

# Exibir o resumo do modelo final
model.summary()

# Definindo a taxa de aprendizado da CNN
taxa_de_aprendizado = 0.0001

# Compilando o modelo
model.compile(Adam(learning_rate=taxa_de_aprendizado),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Treinando o modelo
history = model.fit(
    train_data,
    epochs=30, # Quantidade de épocas
    validation_data=val_data,
#    callbacks=[early_stop]
)

# Salvando a estrutura do modelo
model_structure = model.to_json()
with open('modelo_classificacao_pessoas.json', 'w') as json_file:
    json_file.write(model_structure)

# Salvando os pesos do modelo
model.save_weights('modelo_classificacao_pessoas.weights.h5')

# Salvando os rótulos em um arquivo TXT
labels = train_data.class_indices  # Obter os rótulos do dataset
with open('rotulos.txt', 'w') as txt_file:
    for label, index in labels.items():
        txt_file.write(f"{label}: {index}\n")  # Salva o rótulo e seu índice


Found 3200 images belonging to 1 classes.
Found 800 images belonging to 1 classes.


C:\Users\Felipe Monteiro\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 4, 4, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,572,544 (6.00 MB)

...

 Trainable params: 6,982,804 (26.64 MB)

 Non-trainable params: 1,984 (7.75 KB)

C:\Users\Felipe Monteiro\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 20)

In [10]:
---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
Cell In[3], line 106
    101 model.compile(Adam(learning_rate=taxa_de_aprendizado),
    102               loss='categorical_crossentropy',
    103               metrics=['accuracy'])
    105 # Treinando o modelo
--> 106 history = model.fit(
    107     train_data,
    108     epochs=30,  # Quantidade de épocas
    109     validation_data=val_data
    110 )
    112 # Salvando a estrutura do modelo
    113 model_structure = model.to_json()

File ~\AppData\Roaming\Python\Python312\site-packages\keras\src\utils\traceback_utils.py:122, in filter_traceback.<locals>.error_handler(*args, **kwargs)
    119     filtered_tb = _process_traceback_frames(e.__traceback__)
    120     # To get the full stack trace, call:
    121     # `keras.config.disable_traceback_filtering()`
--> 122     raise e.with_traceback(filtered_tb) from None
    123 finally:
    124     del filtered_tb

File ~\AppData\Roaming\Python\Python312\site-packages\keras\src\utils\module_utils.py:26, in LazyModule.initialize(self)
     24     self.module = importlib.import_module(self.name)
...
     27         f"This requires the {self.name} module. "
     28         f"You can install it via `pip install {self.pip_name}`"
     29     )

ImportError: This requires the scipy module. You can install it via `pip install scipy`
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

SyntaxError: invalid syntax (4289079589.py, line 1)

In [8]:
!pip install scipy


Defaulting to user installation because normal site-packages is not writeable
